In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import VotingRegressor
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings("ignore")
import joblib

In [2]:
def fetch_crypto_data(crypto_pair, start_date):
    """
    Fetches daily historical data for a specified cryptocurrency pair from the CryptoCompare API.

    Parameters:
    - crypto_pair (str): Cryptocurrency pair in the format "BTC/USD".
    - start_date (str): Start date in "YYYY-MM-DD" format.

    Returns:
    - DataFrame: Contains Date, Open, High, Low, and Close prices.
    """
    # Convert start_date to timestamp
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())

    # Parse crypto_pair into individual symbols
    fsym, tsym = crypto_pair.split('/')

    # Define API endpoint and parameters
    url = "https://min-api.cryptocompare.com/data/v2/histoday"
    params = {
        'fsym': fsym,            # From symbol
        'tsym': tsym,            # To symbol
        'limit': 2000,           # Max days to fetch in one request
        'toTs': start_timestamp  # End timestamp (fetches data from start date onward)
    }

    # API Key (register at CryptoCompare to get one if required)
    api_key = "af387748d6eaba1f35e3d283cad9dcaf8dc7504039d637dc71fbcd9874bcd630"
    headers = {
        'authorization': f'Apikey {api_key}'
    }

    # Send request
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    # Check for errors in response
    if data['Response'] != 'Success':
        raise Exception("Error fetching data from CryptoCompare:", data['Message'])

    # Extract data
    historical_data = data['Data']['Data']
    # Convert to DataFrame
    df = pd.DataFrame(historical_data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    df = df[['Date', 'Open', 'High', 'Low', 'Close']]

    return df

In [3]:
data = fetch_crypto_data("DOT/USD", "2024-01-01")

In [4]:
data.to_excel("DOT-USD.xlsx")

In [5]:
def calculate_metrics(data, variable1, variable2):
    """
    Calculates historical and future metrics for a given DataFrame with cryptocurrency data.

    Parameters:
    - data (DataFrame): The historical cryptocurrency data containing 'Date', 'Open', 'High', 'Low', 'Close' columns.
    - variable1 (int): The look-back period for historical high and low metrics.
    - variable2 (int): The look-forward period for future high and low metrics.

    Returns:
    - DataFrame: The input DataFrame with added metric columns.
    """

    # Historical High and Low prices over the last `variable1` days
    data[f'High_Last_{variable1}_Days'] = data['High'].rolling(window=variable1, min_periods=1).max()
    data[f'Low_Last_{variable1}_Days'] = data['Low'].rolling(window=variable1, min_periods=1).min()

    # Days since Historical High and Low
    data[f'Days_Since_High_Last_{variable1}_Days'] = data.apply(
        lambda row: (row['Date'] - data.loc[:row.name, 'Date'][data['High'][:row.name + 1] == row[f'High_Last_{variable1}_Days']].iloc[-1]).days
        if pd.notna(row[f'High_Last_{variable1}_Days']) else np.nan, axis=1
    )
    data[f'Days_Since_Low_Last_{variable1}_Days'] = data.apply(
        lambda row: (row['Date'] - data.loc[:row.name, 'Date'][data['Low'][:row.name + 1] == row[f'Low_Last_{variable1}_Days']].iloc[-1]).days
        if pd.notna(row[f'Low_Last_{variable1}_Days']) else np.nan, axis=1
    )

    # Percentage difference from Historical High and Low
    data[f'%_Diff_From_High_Last_{variable1}_Days'] = (data['Close'] - data[f'High_Last_{variable1}_Days']) / data[f'High_Last_{variable1}_Days'] * 100
    data[f'%_Diff_From_Low_Last_{variable1}_Days'] = (data['Close'] - data[f'Low_Last_{variable1}_Days']) / data[f'Low_Last_{variable1}_Days'] * 100

    # Future High and Low prices over the next `variable2` days
    data[f'High_Next_{variable2}_Days'] = data['High'].shift(-variable2).rolling(window=variable2, min_periods=1).max()
    data[f'Low_Next_{variable2}_Days'] = data['Low'].shift(-variable2).rolling(window=variable2, min_periods=1).min()

    # Percentage difference from Future High and Low
    data[f'%_Diff_From_High_Next_{variable2}_Days'] = (data['Close'] - data[f'High_Next_{variable2}_Days']) / data[f'High_Next_{variable2}_Days'] * 100
    data[f'%_Diff_From_Low_Next_{variable2}_Days'] = (data['Close'] - data[f'Low_Next_{variable2}_Days']) / data[f'Low_Next_{variable2}_Days'] * 100

    return data

In [6]:
data['Date'] = pd.to_datetime(data['Date'])
variable1, variable2 = 7, 5
data = calculate_metrics(data, variable1, variable2)
#data.to_csv("crypto_data1.csv", index=False)

In [7]:
data

,Date,Open,High,Low,Close,High_Last_7_Days,Low_Last_7_Days,Days_Since_High_Last_7_Days,Days_Since_Low_Last_7_Days,%_Diff_From_High_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,Low_Next_5_Days,%_Diff_From_High_Next_5_Days,%_Diff_From_Low_Next_5_Days
0,2018-07-11,0.000,0.000,0.000,0.000,0.000,0.000,0,0,NaN,NaN,0.000,0.000,NaN,NaN
1,2018-07-12,0.000,0.000,0.000,0.000,0.000,0.000,0,0,NaN,NaN,0.000,0.000,NaN,NaN
2,2018-07-13,0.000,0.000,0.000,0.000,0.000,0.000,0,0,NaN,NaN,0.000,0.000,NaN,NaN
3,2018-07-14,0.000,0.000,0.000,0.000,0.000,0.000,0,0,NaN,NaN,0.000,0.000,NaN,NaN
4,2018-07-15,0.000,0.000,0.000,0.000,0.000,0.000,0,0,NaN,NaN,0.000,0.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,2023-12-28,8.702,8.918,8.311,8.436,9.587,7.598,2,5,-12.005841,11.029218,8.689,8.039,-2.911727,4.938425
1997,2023-12-29,8.436,8.689,8.087,8.295,9.587,7.598,3,6,-13.476583,9.173467,8.645,8.039,-4.048583,3.184476
1998,2023-12-30,8.295,8.487,8.098,8.354,9.587,8.087,4,1,-12.861166,3.301595,8.645,8.039,-3.366108,3.918398
1999,2023-12-31,8.354,8.645,8.039,8.201,9.587,8.039,5,0,-14.457077,2.015176,8.629,8.064,-4.960019,1.698909


In [8]:
def fetch_crypto_data(crypto_pair, start_date, end_date):
    url = "https://min-api.cryptocompare.com/data/v2/histoday"
    fsym, tsym = crypto_pair.split('/')
    params = {
        'fsym': fsym,
        'tsym': tsym,
        'limit': 2000,
        'toTs': pd.to_datetime(end_date).timestamp(),
    }
    headers = {'authorization': 'Apikey YOUR_API_KEY'}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    df = pd.DataFrame(data['Data']['Data'])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    df = df[['Date', 'Open', 'High', 'Low', 'Close']]
    return df

In [9]:
# Usage Example
#data = fetch_crypto_data("BTC/USD", "2020-01-01", "2024-01-01")
#data.to_csv("crypto_data.csv", index=False)

In [10]:
df1 = pd.read_csv("/content/crypto_data1.csv")

In [11]:
def train_model(data):
    """
    Trains a machine learning model to predict future price differences.

    Parameters:
    - data (DataFrame): The DataFrame containing calculated metrics and target variables.

    Returns:
    - model: The trained machine learning model.
    - metrics: A dictionary with model evaluation metrics (RMSE and R-squared).
    """

    # Define the input features and target variables
    feature_cols = [
        f'Days_Since_High_Last_{variable1}_Days',
        f'%_Diff_From_High_Last_{variable1}_Days',
        f'Days_Since_Low_Last_{variable1}_Days',
        f'%_Diff_From_Low_Last_{variable1}_Days'
    ]
    target_cols = [
        f'%_Diff_From_High_Next_{variable2}_Days',
        f'%_Diff_From_Low_Next_{variable2}_Days'
    ]

    # Drop rows with NaN values in target columns
    data = data.dropna(subset=target_cols)

    # Separate features and target variables
    X = data[feature_cols]
    y = data[target_cols]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the model (RandomForestRegressor)
    model = MultiOutputRegressor(LinearRegression())
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    # Save the model
    joblib.dump(model, 'trained_model.pkl')

    # Return model and evaluation metrics
    metrics = {"RMSE": rmse, "R2_Score": r2}
    return model, metrics

def predict_outcomes(model, input_features):
    """
    Uses the trained model to predict future price differences.

    Parameters:
    - model: The trained machine learning model.
    - input_features (list or array): The list of input feature values for prediction.

    Returns:
    - predictions: A dictionary with predicted values for target variables.
    """

    # Make predictions
    prediction = model.predict([input_features])
    predictions = {
        f'%_Diff_From_High_Next_{variable2}_Days': prediction[0][0],
        f'%_Diff_From_Low_Next_{variable2}_Days': prediction[0][1]
    }
    return predictions


In [12]:
data = pd.read_csv('/content/crypto_data1.csv')

# Train the model
model, metrics = train_model(data)
print("Model Metrics:", metrics)

# Predict using new data
model = joblib.load('trained_model.pkl')
new_data = [7, -3.2, 5, 2.5]  # Example input feature values
predictions = predict_outcomes(model, new_data)
print("Predicted Outcomes:", predictions)

Model Metrics: {'RMSE': 6.283251564166778, 'R2_Score': 0.10750164002639623}
Predicted Outcomes: {'%_Diff_From_High_Next_5_Days': -4.018364369449523, '%_Diff_From_Low_Next_5_Days': 3.372373319727212}
